In [1]:
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import os

In [13]:
wiki_data = pd.read_csv("/data/iivanova-23/data/wiki/wiki_quotes.csv")
# train, val = train_test_split(data, test_size=0.3, stratify=data['label'], random_state=42, shuffle=True)
wiki_data['label'].value_counts()
unique_authors = wiki_data['label'].unique()
print(len(unique_authors))

37


In [14]:
data_in_the_wild = pd.read_csv("/data/iivanova-23/data/inthewild/inthewild_transcriptions_final.csv")
data_in_the_wild['label'].value_counts()
unique_authors_inthewild = data_in_the_wild['label'].unique()
print(len(unique_authors_inthewild))
print(set(unique_authors_inthewild) - set(unique_authors))

54
{34, 38, 7, 39, 10, 42, 43, 45, 14, 46, 17, 20, 52, 24, 25, 26, 31}


In [21]:
labels = [37, 28, 51, 0, 8, 49, 16, 34, 38, 7, 39, 10, 42, 43, 45, 14, 46, 17, 20, 52, 24, 25, 26, 31]
data_wild_small = data_in_the_wild[data_in_the_wild['label'].isin(labels)]
data_concat = pd.concat([wiki_data, data_wild_small])
data_concat.to_csv("/data/iivanova-23/data/wild_wiki.csv", index=False)

In [20]:
train, val = train_test_split(data_concat, test_size=0.3, stratify=data_concat['label'], random_state=42, shuffle=True)


In [20]:
meta_data = pd.read_csv('/data/amathur-23/DADA/InTheWild/release_in_the_wild/meta.csv')
meta_data['speaker'] = meta_data['speaker'].replace('JFK', 'John F. Kennedy').replace('FDR', 'Franklin D. Roosevelt').replace('Harry Truman', 'Harry S. Truman').replace('Mr. Rogers','Fred Rogers')
author_id_map = {author: i for i, author in enumerate(meta_data['speaker'].unique())}
author_id_map

{'Alec Guinness': 0,
 'Barack Obama': 1,
 'Christopher Hitchens': 2,
 'Ayn Rand': 3,
 'Donald Trump': 4,
 'Bernie Sanders': 5,
 'Calvin Coolidge': 6,
 'Winston Churchill': 7,
 'Bob Ross': 8,
 'Richard Nixon': 9,
 '2Pac': 10,
 'Ronald Reagan': 11,
 'John F. Kennedy': 12,
 'Arnold Schwarzenegger': 13,
 'Nick Offerman': 14,
 'Bill Clinton': 15,
 'Martin Luther King': 16,
 'Bill Burr': 17,
 'Mark Zuckerberg': 18,
 'George W. Bush': 19,
 'Dave Chappelle': 20,
 'Boris Johnson': 21,
 'Alan Watts': 22,
 'Louis C.K.': 23,
 'The Notorious B.I.G.': 24,
 'Jimmy Carter': 25,
 'Dwight Eisenhower': 26,
 'Mitch Hedberg': 27,
 'Alexandria Ocasio-Cortez': 28,
 'Milton Friedman': 29,
 'Franklin D. Roosevelt': 30,
 'Queen Elizabeth II': 31,
 'Jerry Seinfeld': 32,
 'Tucker Carlson': 33,
 'Nelson Mandela': 34,
 'John Cleese': 35,
 'Frank Sinatra': 36,
 'Louis Farrakhan': 37,
 'Norm MacDonald': 38,
 'Gilbert Gottfried': 39,
 'George Carlin': 40,
 'Malcolm X': 41,
 'Robert Kardashian': 42,
 'Kamala Harris': 4

In [25]:
author_id_map['Winston Churchill']

7

In [ ]:
in_the_wild = pd.read_csv("/data/iivanova-23/data/inthewild/inthewild_transcriptions_final.csv")
# real = in_the_wild[in_the_wild['type'] == 'bona-fide']
# len(real) 
# # Check for NaN values in the 'label' column
# nan_labels = in_the_wild[in_the_wild['label'].isna()]
# print("Rows with NaN labels:")
# print(nan_labels)
# in_the_wild['label'] = in_the_wild['author_name'].map(author_id_map)

# Proceed with train_test_split if there are no NaN values
# if nan_labels.empty:
#     test, train = train_test_split(real, test_size=0.3, random_state=42, stratify=real['label'])
# else:
#     print("Cannot perform train_test_split due to NaN values in 'label' column.") 


Rows with NaN labels:
            file         author_name       type  \
16        16.wav   Winston Churchill      spoof   
17        17.wav   Winston Churchill      spoof   
22        22.wav                2Pac  bona-fide   
28        28.wav       Nick Offerman  bona-fide   
31        31.wav           Bill Burr      spoof   
...          ...                 ...        ...   
31752  31755.wav   Winston Churchill      spoof   
31753  31756.wav  Queen Elizabeth II      spoof   
31762  31765.wav   Winston Churchill  bona-fide   
31763  31766.wav   Robert Kardashian  bona-fide   
31768  31771.wav   Winston Churchill      spoof   

                                                    text  index  tokens  label  
16           We all bleed the same regrowth of patriots.     17       8    NaN  
17               and spent trillions of dollars overseas     18       6    NaN  
22                 lending money to everybody except us.     23       6    NaN  
28                                       

In [38]:
in_the_wild['label'] = in_the_wild['author_name'].map(author_id_map)
real = in_the_wild[in_the_wild['type'] == 'bona-fide']
train, test = train_test_split(real, test_size=0.3, random_state=42, stratify=real['label'])

In [41]:
np.sort(real['label'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53])

In [ ]:
# in_the_wild.to_csv('/data/iivanova-23/data/inthewild/inthewild_transcriptions_final.csv', index=False)

## Helper functions

In [3]:
def form_triplets(df):
    triplets = []
    for i, row in df.iterrows():
        anchor = row
        anchor_label = row['label']
        positive_sample = df[df['label'] == anchor_label].sample(n=1).iloc[0]
        negative_sample = df[df['label'] != anchor_label].sample(n=1).iloc[0]
        triplets.append((anchor['index'], positive_sample['index'], negative_sample['index']))
    triplets = [(int(a), int(p), int(n)) for a, p, n in triplets]
    return triplets

## VoxCeleb2

In [9]:
data = '/data/amathur-23/DADA/VoxCeleb2/'
train = os.path.join(data, 'train')
train_0 = os.listdir(train)[0]
train_data = []
with open(os.path.join(train, train_0)) as f:
    file_data = json.load(f)
    train_data.append([file_data['transcription'], file_data['speaker_id'], file_data['gender']])
    

In [11]:
train_df = pd.DataFrame(train_data, columns=['transcription', 'speaker_id', 'gender'])  
train_df

,transcription,speaker_id,gender
0,"like I'm divorced, but now he wants me to co...",id00649,female


In [9]:
import pandas as pd
data = '/data/iivanova-23/data/VoxCeleb2/val/meta.csv'
data = pd.read_csv(data)
data['file_path'] = data['file_path'].str.replace('/amathur-23/DADA/', '/iivanova-23/data/')
data.to_csv('/data/iivanova-23/data/VoxCeleb2/val/meta.csv', index=False)


In [31]:
data = '/data/amathur-23/DADA/VoxCeleb2/train/meta.csv'
meta_df = pd.read_csv(data)
len(meta_df['speaker_id'].unique())
samples_per_speaker = meta_df['speaker_id'].value_counts()
top_100_speakers = samples_per_speaker.head(10).index
meta_df_small = meta_df[meta_df['speaker_id'].isin(top_100_speakers)]
unique_speakers = meta_df_small['speaker_id'].unique()

In [39]:
data = '/data/amathur-23/DADA/VoxCeleb2/'
train = os.path.join(data, 'train')
train_data = []
label_speaker_map = {}
for file in os.listdir(train)[:5000]:
    with open(os.path.join(train, file), 'r') as f: 
        file_data = json.load(f)
        train_data.append([file_data['transcription'], file_data['speaker_id'], file_data['gender']])
train_df = pd.DataFrame(train_data, columns=['transcription', 'speaker_id', 'gender'])  

## ASVspoof 2021 

In [14]:
csv_file = pd.read_csv('/home/infres/iivanova-23/DADA/iivanova-23/data/asvspoof2021/transcriptions_gpu.csv')
print(len(csv_file[csv_file['type']=='spoof']))
print(len(csv_file[csv_file['type']!='spoof']))
spoof_small = csv_file[csv_file['type']=='spoof'].sample(3000)
bonafide_small = csv_file[csv_file['type']!='spoof']
data_asv = pd.concat([spoof_small, bonafide_small])
data_asv['index'] = range(1, len(data_asv) + 1)
data_asv['tokens'] = data_asv['transcription'].apply(lambda x: len(x.split()))
print(data_asv['tokens'].describe())
data_asv.to_csv('/home/infres/iivanova-23/DADA/iivanova-23/data/asvspoof2021/transcriptions_small.csv', index=False)


51998
1934
count    4934.000000
mean        8.572152
std         6.037349
min         1.000000
25%         6.000000
50%         8.000000
75%        10.000000
max       333.000000
Name: tokens, dtype: float64


## In The Wild

In [10]:
meta_in_the_wild = pd.read_csv('/data/amathur-23/DADA/InTheWild/release_in_the_wild/meta.csv')
unique_authors_meta = meta_in_the_wild['speaker'].unique()
meta_in_the_wild['speaker'] = meta_in_the_wild['speaker'].replace('JFK', 'John F. Kennedy').replace('FDR', 'Franklin D. Roosevelt').replace('Harry Truman', 'Harry S. Truman').replace('Mr. Rogers','Fred Rogers')

In [53]:

from sklearn.model_selection import train_test_split
data_wiki = pd.read_csv('~/DADA/Data/WikiQuotes_train.csv')
# data['index'] = range(1, len(data) + 1)
# data = data.to_csv('~/DADA/Data/WikiQuotes_train.csv', index=False)
# data_wiki = data_wiki[data_wiki['type'] != 'spoof']
# data_authors = data_wiki[data_wiki['label'] < 10]
# data_wild = pd.read_csv('/data/iivanova-23/DataDADA/wild_transcription_meta.csv')
# data_wild['tokens'] = data_wild['text'].apply(lambda x: len(x.split()))
# data_wild = data_wild[(data_wild['tokens'] > 5) & (data_wild['tokens'] < 40)]
# data_wild = data_wild[data_wild['label'] < 10]
# data_merged = pd.concat([data_authors, data_wild])
# # len(data_merged)
# data_real = data_merged[data_merged['type'] != 'spoof']
# data_spoof = data_merged[data_merged['type'] == 'spoof']
# train, test = train_test_split(data_real, test_size=0.3, random_state=42)
# data_merged.to_csv('/data/iivanova-23/DataDADA/train_merged.csv', index=False)

### In the wild dataset

In [3]:
def find_unique_authors_create_dataFrame(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    unique_authors = set()
    authors_list = []
    ids_list = []
    quotes_list = []
    types_list = []
    author_id_map = {}
    current_id = 0
    for item in data:
        author = item.get('speaker')
        if author:
            if author not in author_id_map:
                    author_id_map[author] = current_id
                    current_id += 1
            unique_authors.add(author)
            authors_list.append(author)
            ids_list.append(author_id_map[author])
            quotes_list.append(item.get('content'))
            types_list.append(item.get('label'))
    data = pd.DataFrame({
        'author_id': ids_list,
        'author_name': authors_list,
        'text': quotes_list,
        'type': types_list,
        'length': [len(quote) for quote in quotes_list]
    })
    data['author_name'] = data['author_name'].replace('JFK', 'John F. Kennedy').replace('FDR', 'Franklin D. Roosevelt').replace('Harry Truman', 'Harry S. Truman').replace('Mr. Rogers','Fred Rogers')
    author_id_map['John F. Kennedy'] = author_id_map.pop('JFK')
    author_id_map['Franklin D. Roosevelt'] = author_id_map.pop('FDR')
    author_id_map['Harry S. Truman'] = author_id_map.pop('Harry Truman')
    author_id_map['Fred Rogers'] = author_id_map.pop('Mr. Rogers')
    return unique_authors, data, author_id_map

In [14]:
def plot_author_quotes_count(data):
    quote_counts = data['author_name'].value_counts()
    plt.figure(figsize=(10, 8))
    quote_counts.plot(kind='bar', color=plt.cm.viridis(np.linspace(0, 1, len(quote_counts))))
    plt.title('Number of Quotes per Author')
    plt.xlabel('Author')
    plt.ylabel('Number of Quotes')
    plt.xticks(rotation=90)
    plt.show()
    return quote_counts

### Wikiquotes

In [ ]:
def tokens_count(data):
    data['tokens'] = data['text'].apply(lambda x: len(x.split()))
    return data

def plot_tokens_distribution(data):
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='author_name', y='tokens', data=data, palette='blend:#7AB,#EDA', hue='author_name')
    plt.title('Tokens Distribution per Author')
    plt.xlabel('Author')
    plt.ylabel('Number of Tokens')
    plt.xticks(rotation=90)
    plt.show()


In [ ]:
wild_data = pd.read_csv('/data/iivanova-23/data/wild_transcription_meta.csv')
wild_data = wild_data.drop(columns=['author_id', 'index', 'label'])
wild_data['tokens'] = wild_data['text'].apply(lambda x: len(x.split()))
wild_data = wild_data[~wild_data['author_name'].isin(['Jimmy Carter', 'Dave Chappelle'])]

In [192]:
labels = wild_data['author_name'].unique()
author_id_map = {author: i for i, author in enumerate(labels)}
wild_data['label'] = wild_data['author_name'].map(author_id_map)

In [7]:
wiki_quotes = pd.read_csv('/data/iivanova-23/data/wiki/wiki_quotes.csv')
author_id_map = wiki_quotes[['author_name', 'label']].drop_duplicates().set_index('author_name').to_dict()['label']

In [13]:
meta_in_the_wild = pd.read_csv('/data/amathur-23/DADA/InTheWild/release_in_the_wild/meta.csv')
meta_in_the_wild['speaker'] = meta_in_the_wild['speaker'].replace('JFK', 'John F. Kennedy').replace('FDR', 'Franklin D. Roosevelt').replace('Harry Truman', 'Harry S. Truman').replace('Mr. Rogers','Fred Rogers')
unique_authors_meta = meta_in_the_wild['speaker'].unique()

transciptions = pd.read_csv('/home/infres/iivanova-23/DADA/iivanova-23/data/inthewild/in_the_wild_transcriptions.csv')
transciptions['speaker'] = transciptions['speaker'].replace('JFK', 'John F. Kennedy').replace('FDR', 'Franklin D. Roosevelt').replace('Harry Truman', 'Harry S. Truman').replace('Mr. Rogers','Fred Rogers')
transciptions = transciptions.rename(columns={'label': 'type'})
transciptions = transciptions.rename(columns={'speaker': 'author_name'})
transciptions = transciptions.rename(columns={'transcription': 'text'})
transciptions['label']  = transciptions['author_name'].map(author_id_map)
transciptions.to_csv('/data/iivanova-23/data/inthewild/inthewild_transcriptions_final.csv', index=False)


## Wikiquotes only

In [54]:
import os
folder_path = '/data/iivanova-23/data/'
metadata_file = os.path.join(folder_path, 'quotes-500-en-sources.csv')
metadata_df = pd.read_csv(metadata_file)
quotes_file = os.path.join(folder_path, 'quotes-500-en-quotes.csv')
quotes_df = pd.read_csv(quotes_file)
filtered_quotes_df = quotes_df[quotes_df['source_id'].isin(metadata_df[metadata_df['name'].isin(author_id_map.keys())]['source_id'])]
source_id_to_name = dict(zip(metadata_df['source_id'], metadata_df['name']))
filtered_quotes_df['author_name'] = filtered_quotes_df['source_id'].map(source_id_to_name)
wiki_data = pd.DataFrame({
    'label': filtered_quotes_df['author_name'].map(author_id_map),
    'author_name': filtered_quotes_df['author_name'],
    'text': filtered_quotes_df['quote'],
    'type': 'bona-fide',
    'length': filtered_quotes_df['quote'].apply(len),
    'tokens': filtered_quotes_df['quote'].apply(lambda x: len(x.split()))
})

/tmp/ipykernel_643076/3898041049.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_quotes_df['author_name'] = filtered_quotes_df['source_id'].map(source_id_to_name)


In [55]:
def truncate_data(data, max_tokens):
    data_new = []
    for i in range(len(data)):
        text = data.iloc[i]['text']
        words = text.split()
        tokens = data.iloc[i]['tokens']
        for j in range(0, tokens, max_tokens):
            chunk = ' '.join(words[j:j + max_tokens])
            # print(chunk, len(chunk.split()))
            if len(chunk.split()) > 5:
                data_new.append({
                    'text': chunk,
                    'author_name': data.iloc[i]['author_name'],
                    'tokens': len(chunk.split()), 
                    'label': data.iloc[i]['label'],
                    'length': len(chunk), 
                    'type': data.iloc[i]['type'],
                    
                })

    df = pd.DataFrame(data_new)
    return df

wiki_quotes_truncated = truncate_data(wiki_data, 64)
wiki_quotes_truncated['index'] = range(1, len(wiki_quotes_truncated) + 1)
 
wiki_quotes_truncated.to_csv('/data/iivanova-23/data/wiki/wiki_quotes.csv', index=False)
train, test = train_test_split(wiki_quotes_truncated, test_size=0.3, random_state=42)
train_triples = form_triplets(train)
test_triples = form_triplets(test)
# spoofed_triplets = form_triplets(all_data_spoof)
print(len(train_triples), len(test_triples))

data_json = {'train': train_triples, 'test': test_triples}

with open('/data/iivanova-23/data/wiki/meta_wiki.json', 'w') as f:
    json.dump(data_json, f)

2344 1005
